![penguins](https://raw.githubusercontent.com/allisonhorst/palmerpenguins/master/man/figures/lter_penguins.png)

https://github.com/allisonhorst/palmerpenguins

In [ ]:
!pip install -r requirements.txt

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

penguins = pd.read_csv("penguins.csv")

penguins.head()

To check column datatypes

In [ ]:
print(penguins[e].dtype for e in penguins.columns)

To summarize: 


species - cualitative

island - cualitative

bill_length_mm - real cuantitative

bill_depth_mm - real cuantitative

flipper_length_mm - real cuantitative

body_mass_g - real cuantitative

sex - cualitative

Class unbalance: 

We'll turn our attention to cualitative classes first

In [ ]:
[penguins[e].value_counts() for e in penguins.columns if not is_numeric_dtype(penguins[e].dtype)]

As we can see, there's no heavy unbalance except on the case of the island, where we have quite the mayhem because an insland only takes slightly more than the 10% of the total registries

Before getting on the variable itself, it's worth noting something: 

In [ ]:
[f"{penguins[penguins.island == e].species.value_counts().index} on island {e}" for e in penguins.island.value_counts().index]

As we can see, only a single species inhabits the three islands, which might not match reality, as penguins are pretty nomadic (worse in the case of the Gentoo species, which are barely territorial),, so we can drop this variable down

In [ ]:
penguins.drop(axis=1, columns="island", inplace=True)

Variable exploration

Now, onto the variables themselves. First we want to see if there's any correlation

In [ ]:
sns.heatmap(penguins.corr(), annot=True, cbar=True, cmap="RdYlGn")

As we can see, the flipper length is heavily correlated with the weight, so we can drop it

In [ ]:
penguins.drop(axis=1, columns="flipper_length_mm", inplace=True)

Variations due to sexual dimorphism can be an issue, depending on how heavy they are, but as we can see in the following graphics, there are heavy, clear differences between the two sexes (instead of every individual being more centered on the average, no matter of the gender, which would us allow to drop it):

In [ ]:
plt.show([sns.pairplot(penguins[penguins.species == e], hue='sex') for e in penguins.species.value_counts().index])

Preprocessing

First of all, we'll seek all registries with null values

In [ ]:
[f"Percent of not nulls in column {e} is {len([i for i in penguins[e].notnull() if i])/len(penguins)}" for e in penguins.columns]

As you can see, on average, the percent of null fields is quite small, so we can get rid of all of them right away

In [ ]:
penguins.dropna(inplace=True)

Now we get into muddy terrain. Here we used for the standard deviation zero degrees of freedom (ddof), as we are assuming our set to be a population, not a sample, thus forcing it to take zero instead of the default one.

Heavy variations of the standard deviation with the average mightmark the presence of outliers (as a quick reference), so we shall do:

In [ ]:
from pandas.api.types import is_numeric_dtype

[f"Column {e} variation is {penguins[e].std(ddof=0)/penguins[e].mean()}" for e in penguins.columns if is_numeric_dtype(penguins[e].dtype)]

Every column is within normalcy except for the body mass one. As such, we must proceed with further analysis. For this we use the z-score, which for a certain value marks how many times the standard deviation is that value deviated from the average. Example gratia, a z-score of 2, would mean that the value minus the average is twice the standard deviation.

Also, outliers appear as greater or equal to three times the standard deviation or lesser or equal to minus three times the standard deviation. We can simplify it by saying that its absolute z-score is greater or equal to three. Now, let's check the evolution of the (absolute) z-scores:

In [ ]:
from scipy.stats import zscore

mass = penguins.mass

for e in np.arange(0., 3., 0.25):
    print(f"Number of elements over {e} times the deviation is {len([i for i in abs(zscore(mass, ddof=0)) if i > e])}")

As we can see, there's no values with an z-score greater than or equal to three, so there are no outliers. Plus, its evolution is pretty good, following the trace of a normal distribution, so it is due to the scale of the value. We can correcting it by normalizing the values


In [ ]:
from sklearn import preprocessing as prepro

normalizer = prepro.Normalizer(norm='l1')
norm_mass = normalizer.fit_transform(mass.values.reshape(1, -1))[0]

penguins["body_mass_g"] = norm_mass

To end the preprocessing, we will convert all cualitative columns into labels

In [ ]:
species = penguins.species
sex = penguins.sex

le = prepro.LabelEncoder

species_labeled = le.fit_transform(le, y=species)
penguins.species = species_labeled

sex_labeled = le.fit_transform(le, y=sex)
penguins.sex = sex_labeled

Now onto the models.


Before we get into it we'll fetch the basics, plus a pretty nifty function to give us a degree of commodity. Said function will tell us straight away the accuracy of our model and the confusion matrix (which tells us how many positives on the predictions have scored/missed the mark)

We'll also set the basic sets for the model training

In [ ]:
from sklearn.metrics import accuracy_score, plot_confusion_matrix, classification_report, confusion_matrix

def show_metrics(clf, y_test, y_pred):
    print(f'Accuracy score: {int(accuracy_score(y_test, y_pred)*100)}%\n')
    print(classification_report(y_test, y_pred))
    
    disp = plot_confusion_matrix(clf, X_test, y_test)
    disp.figure_.suptitle("Confusion Matrix")
    plt.show()

from sklearn.model_selection import train_test_split

X = penguins.drop("species", axis=1)
Y = penguins.species.values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=73)


Now, this is a classification problem (we want to give a label to new  data). Given the map the guys at sklearn kindly give to us (https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html), we'll choose a linear support vector column model, given that we have prelabeled data and our set is greater than 100 entries. Allow us to train the model, and check the data:

In [ ]:
from sklearn.svm import SVC, LinearSVC

linear_svc = SVC(gamma='auto')
linear_svc.fit(X_train, Y_train)

y_pred = linear_svc.predict(X_test)
show_metrics(linear_svc, Y_test, Y_pred)


Which, as we can see has a good precision with an acceptable confusion matrix.

Now, we want to check how effective was this, so we do cross-validation tests with different kernels

In [ ]:
from sklearn.model_selection import cross_val_score

[print(i) for i in [cross_val_score(SVC(kernel=e, C=1, random_state=73), X, Y, cv=5) for e in ['linear', 'rbf', 'poly']]]

Which gives us pretty good results for the linear one, somewhat good for the rfb and plainly good on the polynomial. As such, our model is well defined, although open to improvements